In [1]:
import gym
import numpy as np

from gym.wrappers import AtariPreprocessing
gym.new_step_api=True
env = gym.make('Pong-v0')

H = 200 # number of hidden layer neurons
D = 80 * 80 # input dimensionality: 80x80 grid
model = {}
model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
model['W2'] = np.random.randn(H) / np.sqrt(H)
# hyperparameters
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-1
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

def sigmoid(x): 
  return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
  I=np.asarray(I)
  I = I[35:195] # crop
  I = I[::2,::2,0] # downsample by factor of 2
  
  I[I == 144] = 0 # erase background (background type 1)
  
  I[I == 109] = 0 # erase background (background type 2)
  
  I[I != 0] = 1 # everything else (paddles, ball) just set to 1
  return I.astype(float).ravel()

def discount_rewards(r):
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size)):
    if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add
  return discounted_r

def policy_forward(x):
  h = np.dot(model['W1'], x)
  h[h<0] = 0 # ReLU nonlinearity
  logp = np.dot(model['W2'], h)
  p = sigmoid(logp)
  return p, h # return probability of taking action 2, and hidden state

def policy_backward(epx, eph, epdlogp):
  """ backward pass. (eph is array of intermediate hidden states) """
  dW2 = np.dot(eph.T, epdlogp).ravel()
  dh = np.outer(epdlogp, model['W2'])
  dh[eph <= 0] = 0 # backpro prelu
  dW1 = np.dot(dh.T, epx)
  return {'W1':dW1, 'W2':dW2}

def model_step(model, observation, prev_x):

  cur_x = prepro(observation)
  x = cur_x - prev_x if prev_x is not None else np.zeros(D)
  prev_x = cur_x
  
  # forward the policy network and sample an action from the returned probability
  aprob, _ = policy_forward(x)
  action = 2 if aprob >= 0.5 else 3 # roll the dice!
  
  return action, prev_x

def play_game(env, model):
  observation = env.reset()

  frames = []
  cumulated_reward = 0

  prev_x = None # used in computing the difference frame

  for t in range(1000):
      frames.append(env.render(mode = 'rgb_array'))
      action, prev_x = model_step(model, observation, prev_x)
      observation, reward, done, info = env.step(action)
      cumulated_reward += reward
      if done:
          print("Episode finished after {} timesteps, accumulated reward = {}".format(t+1, cumulated_reward))
          break
  print("Episode finished without success, accumulated reward = {}".format(cumulated_reward))
  env.close()
  display_frames_as_gif(frames)

def train_model(env, model, total_episodes = 100):
  hist = []
  observation = env.reset()

  prev_x = None # used in computing the difference frame
  xs,hs,dlogps,drs = [],[],[],[]
  running_reward = None
  reward_sum = 0
  episode_number = 0

  while True:
  
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

    # record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    y = 1 if action == 2 else 0 # a "fake label"
    dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

    # step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward

    drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

    if done: # an episode finished
      episode_number += 1

      # stack together all inputs, hidden states, action gradients, and rewards for this episode
      epx = np.vstack(xs)
      eph = np.vstack(hs)
      epdlogp = np.vstack(dlogps)
      epr = np.vstack(drs)
      xs,hs,dlogps,drs = [],[],[],[] # reset array memory

      # compute the discounted reward backwards through time
      discounted_epr = discount_rewards(epr)
      # standardize the rewards to be unit normal (helps control the gradient estimator variance)
      discounted_epr -= np.mean(discounted_epr)
      discounted_epr /= np.std(discounted_epr)

      epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
      grad = policy_backward(epx, eph, epdlogp)
      for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

      # perform rmsprop parameter update every batch_size episodes
      if episode_number % batch_size == 0:
        for k,v in model.items():
          g = grad_buffer[k] # gradient
          rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
          model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
          grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

      # boring book-keeping
      running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
      hist.append((episode_number, reward_sum, running_reward))
      print ('episode %f, reward total was %f. running mean: %f' % (episode_number, reward_sum, running_reward))
      reward_sum = 0
      observation = env.reset() # reset env
      prev_x = None
      if episode_number == total_episodes: 
        return hist

   
    

C:\Users\soura\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\envs\registration.py:592: UserWarning: WARN: The environment Pong-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
C:\Users\soura\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
C:\Users\soura\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [2]:
%time hist1 = train_model(env, model, total_episodes=7000)

C:\Users\soura\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\utils\passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


episode 1.000000, reward total was -21.000000. running mean: -21.000000
episode 2.000000, reward total was -21.000000. running mean: -21.000000
episode 3.000000, reward total was -20.000000. running mean: -20.990000
episode 4.000000, reward total was -20.000000. running mean: -20.980100
episode 5.000000, reward total was -20.000000. running mean: -20.970299
episode 6.000000, reward total was -21.000000. running mean: -20.970596
episode 7.000000, reward total was -21.000000. running mean: -20.970890
episode 8.000000, reward total was -21.000000. running mean: -20.971181
episode 9.000000, reward total was -20.000000. running mean: -20.961469
episode 10.000000, reward total was -21.000000. running mean: -20.961855


C:\Users\soura\AppData\Local\Temp\ipykernel_14664\679866968.py:22: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]


episode 11.000000, reward total was -21.000000. running mean: -20.962236
episode 12.000000, reward total was -21.000000. running mean: -20.962614
episode 13.000000, reward total was -21.000000. running mean: -20.962988
episode 14.000000, reward total was -21.000000. running mean: -20.963358
episode 15.000000, reward total was -21.000000. running mean: -20.963724
episode 16.000000, reward total was -21.000000. running mean: -20.964087
episode 17.000000, reward total was -21.000000. running mean: -20.964446
episode 18.000000, reward total was -21.000000. running mean: -20.964802
episode 19.000000, reward total was -21.000000. running mean: -20.965154
episode 20.000000, reward total was -21.000000. running mean: -20.965502
episode 21.000000, reward total was -21.000000. running mean: -20.965847
episode 22.000000, reward total was -21.000000. running mean: -20.966189
episode 23.000000, reward total was -21.000000. running mean: -20.966527
episode 24.000000, reward total was -21.000000. run

episode 123.000000, reward total was -21.000000. running mean: -20.987748
episode 124.000000, reward total was -21.000000. running mean: -20.987870
episode 125.000000, reward total was -21.000000. running mean: -20.987991
episode 126.000000, reward total was -21.000000. running mean: -20.988112
episode 127.000000, reward total was -21.000000. running mean: -20.988230
episode 128.000000, reward total was -21.000000. running mean: -20.988348
episode 129.000000, reward total was -21.000000. running mean: -20.988465
episode 130.000000, reward total was -21.000000. running mean: -20.988580
episode 131.000000, reward total was -21.000000. running mean: -20.988694
episode 132.000000, reward total was -21.000000. running mean: -20.988807
episode 133.000000, reward total was -21.000000. running mean: -20.988919
episode 134.000000, reward total was -21.000000. running mean: -20.989030
episode 135.000000, reward total was -21.000000. running mean: -20.989140
episode 136.000000, reward total was -

...

episode 6511.000000, reward total was -21.000000. running mean: -21.000000
episode 6512.000000, reward total was -21.000000. running mean: -21.000000
episode 6513.000000, reward total was -21.000000. running mean: -21.000000
episode 6514.000000, reward total was -21.000000. running mean: -21.000000
episode 6515.000000, reward total was -21.000000. running mean: -21.000000
episode 6516.000000, reward total was -21.000000. running mean: -21.000000
episode 6517.000000, reward total was -21.000000. running mean: -21.000000
episode 6518.000000, reward total was -21.000000. running mean: -21.000000
episode 6519.000000, reward total was -21.000000. running mean: -21.000000
episode 6520.000000, reward total was -21.000000. running mean: -21.000000
episode 6521.000000, reward total was -21.000000. running mean: -21.000000
episode 6522.000000, reward total was -21.000000. running mean: -21.000000
episode 6523.000000, reward total was -21.000000. running mean: -21.000000
episode 6524.000000, rewa

episode 6621.000000, reward total was -21.000000. running mean: -21.000000
episode 6622.000000, reward total was -21.000000. running mean: -21.000000
episode 6623.000000, reward total was -21.000000. running mean: -21.000000
episode 6624.000000, reward total was -21.000000. running mean: -21.000000
episode 6625.000000, reward total was -21.000000. running mean: -21.000000
episode 6626.000000, reward total was -21.000000. running mean: -21.000000
episode 6627.000000, reward total was -21.000000. running mean: -21.000000
episode 6628.000000, reward total was -21.000000. running mean: -21.000000
episode 6629.000000, reward total was -21.000000. running mean: -21.000000
episode 6630.000000, reward total was -21.000000. running mean: -21.000000
episode 6631.000000, reward total was -21.000000. running mean: -21.000000
episode 6632.000000, reward total was -21.000000. running mean: -21.000000
episode 6633.000000, reward total was -21.000000. running mean: -21.000000
episode 6634.000000, rewa

episode 6731.000000, reward total was -21.000000. running mean: -21.000000
episode 6732.000000, reward total was -21.000000. running mean: -21.000000
episode 6733.000000, reward total was -21.000000. running mean: -21.000000
episode 6734.000000, reward total was -21.000000. running mean: -21.000000
episode 6735.000000, reward total was -21.000000. running mean: -21.000000
episode 6736.000000, reward total was -21.000000. running mean: -21.000000
episode 6737.000000, reward total was -21.000000. running mean: -21.000000
episode 6738.000000, reward total was -21.000000. running mean: -21.000000
episode 6739.000000, reward total was -21.000000. running mean: -21.000000
episode 6740.000000, reward total was -21.000000. running mean: -21.000000
episode 6741.000000, reward total was -21.000000. running mean: -21.000000
episode 6742.000000, reward total was -21.000000. running mean: -21.000000
episode 6743.000000, reward total was -21.000000. running mean: -21.000000
episode 6744.000000, rewa

episode 6841.000000, reward total was -21.000000. running mean: -21.000000
episode 6842.000000, reward total was -21.000000. running mean: -21.000000
episode 6843.000000, reward total was -21.000000. running mean: -21.000000
episode 6844.000000, reward total was -21.000000. running mean: -21.000000
episode 6845.000000, reward total was -21.000000. running mean: -21.000000
episode 6846.000000, reward total was -21.000000. running mean: -21.000000
episode 6847.000000, reward total was -21.000000. running mean: -21.000000
episode 6848.000000, reward total was -21.000000. running mean: -21.000000
episode 6849.000000, reward total was -21.000000. running mean: -21.000000
episode 6850.000000, reward total was -21.000000. running mean: -21.000000
episode 6851.000000, reward total was -21.000000. running mean: -21.000000
episode 6852.000000, reward total was -21.000000. running mean: -21.000000
episode 6853.000000, reward total was -21.000000. running mean: -21.000000
episode 6854.000000, rewa

episode 6951.000000, reward total was -21.000000. running mean: -21.000000
episode 6952.000000, reward total was -21.000000. running mean: -21.000000
episode 6953.000000, reward total was -21.000000. running mean: -21.000000
episode 6954.000000, reward total was -21.000000. running mean: -21.000000
episode 6955.000000, reward total was -21.000000. running mean: -21.000000
episode 6956.000000, reward total was -21.000000. running mean: -21.000000
episode 6957.000000, reward total was -21.000000. running mean: -21.000000
episode 6958.000000, reward total was -21.000000. running mean: -21.000000
episode 6959.000000, reward total was -21.000000. running mean: -21.000000
episode 6960.000000, reward total was -21.000000. running mean: -21.000000
episode 6961.000000, reward total was -21.000000. running mean: -21.000000
episode 6962.000000, reward total was -21.000000. running mean: -21.000000
episode 6963.000000, reward total was -21.000000. running mean: -21.000000
episode 6964.000000, rewa